# 02 - Feature Engineering

# Install

In [ ]:
# !pip install pyproj

# Librerías

In [8]:
import pandas as pd
import numpy as np
from numpy import radians, sin, cos, arctan2, degrees
from pyproj import Transformer
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Cargar dataset enriquecido

In [9]:
df = pd.read_parquet("../data/processed/rich_dataset.parquet")
print(f"Shape: {df.shape}")
df.head()

Shape: (294608, 20)


,UniqueAnimalID,timestamp,mu_lat,mu_lon,se_mu_x,se_mu_y,adj_lat,adj_lon,prev_lat,prev_lon,prev_time,distance_km,time_diff_hr,velocity_kmh,prev_velocity,acceleration_kmph2,date,daylight_hours,is_polar_night,is_midnight_sun
0,1,1986-07-01 12:00:00,69.7718,-141.3942,9375,9375,69.761663,-141.428310,69.769565,-141.323069,1986-07-01 06:00:00,4.158416,6.0,0.693069,NaN,NaN,1986-07-01,24.0,False,True
1,1,1986-07-01 18:00:00,69.7730,-141.3960,5835,5835,69.772693,-141.403451,69.761663,-141.428310,1986-07-01 12:00:00,1.560623,6.0,0.260104,0.693069,-0.072161,1986-07-01,24.0,False,True
2,1,1986-07-02 00:00:00,69.7738,-141.3834,2517,2517,69.773429,-141.388987,69.772693,-141.403451,1986-07-01 18:00:00,0.564330,6.0,0.094055,0.260104,-0.027675,1986-07-02,24.0,False,True
3,1,1986-07-02 06:00:00,69.7736,-141.3448,5323,5323,69.764930,-141.331186,69.773429,-141.388987,1986-07-02 00:00:00,2.424701,6.0,0.404117,0.094055,0.051677,1986-07-02,24.0,False,True
4,1,1986-07-02 12:00:00,69.7727,-141.2902,8840,8840,69.783728,-141.265125,69.764930,-141.331186,1986-07-02 06:00:00,3.301497,6.0,0.550249,0.404117,0.024355,1986-07-02,24.0,False,True


## 2. Correcciones finales y limpieza

In [4]:
# TODO check si queda algo???? y si eso

In [ ]:
df = df.dropna(subset=["prev_lat", "prev_lon", "velocity_kmh", "acceleration_kmph2"])

## 3. 4. Cargar resumen por osa y merge con _rich_data_
+ `dias_tracking`, `TotalDistance_km`, `distance_per_day`, `AvgLatitude`, `Year`

In [ ]:
resumen_path = "data/processed/resumen_trayectorias.parquet"

# Cargar resumen previamente calculado en EDA
resumen_trayectorias = pd.read_parquet(resumen_path)

print(f"Resumen de trayectorias cargado: {resumen_trayectorias.shape[0]} osas.")
resumen_trayectorias.head()

In [ ]:
df = df.merge(resumen_trayectorias, on="UniqueAnimalID", how="left")

print(f"Merge completado: {df.shape[0]} registros con métricas individuales añadidas.")


### 📎 Nota: resumen por osa ya calculado

Las métricas por osa (`dias_tracking`, `TotalDistance_km`, `distance_per_day`, `AvgLatitude`, `Year`, etc.) 
fueron calculadas previamente durante el análisis exploratorio en `01_aEDA.ipynb`.

Aquí se cargan directamente desde el archivo `resumen_trayectorias.parquet` para evitar duplicación de lógica.


## ~~3. Cargar días de tracking por osa~~

In [ ]:
# dias_tracking = pd.read_parquet("data/processed/dias_tracking.parquet")
# df = df.merge(dias_tracking[["UniqueAnimalID", "dias_tracking"]], on="UniqueAnimalID", how="left")

## 4. ~~Resumen por osa (media de lat, lon, distancia total y por día)~~

In [ ]:
# resumen = df.groupby("UniqueAnimalID").agg({
#     "mu_lat": "mean",
#     "mu_lon": "mean",
#     "distance_km": "sum"
# }).reset_index()

# resumen.columns = ["UniqueAnimalID", "AvgLatitude", "AvgLongitude", "TotalDistance_km"]
# resumen = resumen.merge(dias_tracking[["UniqueAnimalID", "dias_tracking"]], on="UniqueAnimalID")
# resumen["distance_per_day"] = resumen["TotalDistance_km"] / resumen["dias_tracking"]
# resumen.head()

## 5. Asignación de estación del año

In [ ]:
def asignar_estacion(fecha):
    mes = fecha.month
    if mes in [12, 1, 2]:
        return "invierno"
    elif mes in [3, 4, 5]:
        return "primavera"
    elif mes in [6, 7, 8]:
        return "verano"
    else:
        return "otoño"

df["season"] = pd.to_datetime(df["timestamp"]).apply(asignar_estacion)

## 6. Añadir proyección polar

Es una transformación de coordenadas geográficas (lat, lon) a coordenadas cartesianas planas (x, y), optimizada para regiones cercanas al Polo Norte.
Usamos EPSG:3413 → NSIDC Sea Ice Polar Stereographic North, que:
 + Corrige la distorsión en zonas árticas que sufren las proyecciones estándar como Mercator.
 + Permite cálculos precisos de distancia y ángulos cuando las osas se mueven cerca del Polo.

In [ ]:
# Crear transformador para proyección polar
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3413", always_xy=True)

# Inicializar listas
x_coords = []
y_coords = []

# Usar tqdm para ver progreso
for lon, lat in tqdm(zip(df["adj_lon"], df["adj_lat"]), total=len(df), desc="Proyectando coordenadas"):
    x, y = transformer.transform(lon, lat)
    x_coords.append(x)
    y_coords.append(y)

# Asignar columnas al DataFrame
df["x_proj"] = x_coords
df["y_proj"] = y_coords

print("Coordenadas proyectadas (EPSG:3413) añadidas con tqdm.")


## 7. Dirección de movimiento

In [ ]:
def bearing_mc(lat1, lon1, lat2, lon2, se_x1, se_y1, se_x2, se_y2, n=30):
    '''
        Función para dirección con propagación de error (Monte Carlo)
    '''
    bearings = []
    for _ in range(n):
        l1 = np.random.normal(lat1, se_y1)
        o1 = np.random.normal(lon1, se_x1)
        l2 = np.random.normal(lat2, se_y2)
        o2 = np.random.normal(lon2, se_x2)

        φ1, φ2 = radians(l1), radians(l2)
        λ1, λ2 = radians(o1), radians(o2)
        y = sin(λ2 - λ1) * cos(φ2)
        x = cos(φ1) * sin(φ2) - sin(φ1) * cos(φ2) * cos(λ2 - λ1)
        θ = degrees(arctan2(y, x))
        bearings.append((θ + 360) % 360)
    return np.mean(bearings)

# Calcular previos para cambio de dirección
df['prev_lat'] = df.groupby('UniqueAnimalID')['adj_lat'].shift(1)
df['prev_lon'] = df.groupby('UniqueAnimalID')['adj_lon'].shift(1)
df['prev_se_x'] = df.groupby('UniqueAnimalID')['se_mu_x'].shift(1)
df['prev_se_y'] = df.groupby('UniqueAnimalID')['se_mu_y'].shift(1)

# Calcular bearing con error
df['bearing'] = df.progress_apply(
    lambda r: bearing_mc(
        r['prev_lat'], r['prev_lon'], r['adj_lat'], r['adj_lon'],
        r['prev_se_x'], r['prev_se_y'], r['se_mu_x'], r['se_mu_y']
    ) if pd.notnull(r['prev_lat']) else np.nan,
    axis=1
)

# Calcular cambio de dirección
df['prev_bearing'] = df.groupby('UniqueAnimalID')['bearing'].shift(1)
df['bearing_change'] = df.apply(
    lambda r: abs((r['bearing'] - r['prev_bearing'] + 180) % 360 - 180)
    if pd.notnull(r['prev_bearing']) else 0,
    axis=1
)

### Distribución de cambios de dirección (_bearing_change_)

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['bearing_change'], bins=60, kde=True, color="#007acc")

plt.title("Distribución de cambios de dirección (bearing_change)", fontsize=14)
plt.xlabel("Cambio de dirección (°)")
plt.ylabel("Frecuencia")
plt.grid(False)
sns.despine()

plt.show()

    Picos cerca de 0° → movimientos rectos → traveling

    Picos hacia 90°–180° → giros bruscos o looping → foraging o drifting

Esto puede alimentar un modelo o una segmentación posterior por patrón de movimiento 🐾

###  Gráfico: _bearing_change_ por estación del año (_season_)

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(
    data=df,
    x="bearing_change",
    hue="season",
    bins=60,
    kde=True,
    palette="Blues",
    element="step",
    common_norm=False
)

plt.title("Distribución de cambios de dirección por estación", fontsize=14)
plt.xlabel("Cambio de dirección (°)")
plt.ylabel("Frecuencia")
plt.grid(False)
sns.despine()
plt.legend(title="Estación")
plt.show();


En invierno, ¿hay más cambios bruscos (mayor variabilidad)?

En verano, ¿predominan trayectorias rectas?

¿Las curvas se separan? Pista de patrones migratorios estacionales.

## 8. _distance_per_day_ a nivel osa real y diario

Visualizar la actividad diaria real de cada osa

Etiquetar patrones activo / estacionario

Hacer análisis temporal mucho más fino

In [ ]:
# 1. Asegúrate de tener una columna fecha
df["date"] = df["timestamp"].dt.date

# 2. Calcular distancia diaria total por osa
dist_por_dia = df.groupby(["UniqueAnimalID", "date"])["distance_km"].sum().reset_index()
dist_por_dia.rename(columns={"distance_km": "distance_per_day_real"}, inplace=True)

# 3. Merge con el df principal si quieres tenerlo punto a punto
df_final = df.merge(dist_por_dia, on=["UniqueAnimalID", "date"], how="left")

print("`distance_per_day_real` añadida al dataset.")

## 9. Etiquetado de patrones de movimiento: _activo_ vs. _estacionario_

| Etiqueta       | Criterio principal                      |
| -------------- | --------------------------------------- |
| `activo`       | Recorre más que cierto umbral ese día   |
| `estacionario` | Recorre poco (posible descanso, deriva) |

**Cómo elegit umbral?** Opción sencilla y común: usar la mediana o percentil 75 de distance_per_day_real 

In [ ]:
umbral = df_final["distance_per_day_real"].quantile(0.75)

print(f"Umbral para actividad diaria: {umbral:.2f} km/día")

In [ ]:
# Etiquetar en función del umbral
df_final["movement_pattern"] = df_final["distance_per_day_real"].apply(
    lambda d: "activo" if d > umbral else "estacionario"
)

df_final["movement_pattern"].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(
    data=df_final,
    x="season",
    hue="movement_pattern",
    palette="Blues"
)

plt.title("Patrones de movimiento por estación", fontsize=14)
plt.xlabel("Estación del año")
plt.ylabel("Número de registros")
plt.grid(False)
sns.despine()
plt.legend(title="Patrón")
plt.show();

In [ ]:
# Calcular proporciones
prop_df = (
    df_final.groupby(["season", "movement_pattern"])
    .size()
    .reset_index(name="count")
    .pivot(index="season", columns="movement_pattern", values="count")
    .fillna(0)
)

# Normalizar por fila (estación)
prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)

# Gráfico
prop_df.plot(
    kind="bar",
    stacked=True,
    color=["#a6cee3", "#1f78b4"],
    figsize=(10, 5)
)

plt.title("Proporción de patrones de movimiento por estación", fontsize=14)
plt.xlabel("Estación del año")
plt.ylabel("Proporción")
plt.legend(title="Patrón")
plt.grid(False)
sns.despine()
plt.show()

¿Hay más días activos en verano?

¿Aumenta el patrón estacionario en invierno (menos movimiento por hielo o caza pasiva)?

¿Alguna estación tiene cambio de tendencia?

## 10. Integración de Variables Ambientales Externas

Durante el desarrollo del proyecto, en una primera fase se incorporaron variables ambientales como:

- `cdr_seaice_conc` (concentración de hielo marino)
- `temp_surface` (temperatura superficial)
- `wind_speed` (velocidad del viento)
- `cloud_cover` (cobertura nubosa)

 El proceso resultó lento (casi 4 h) y, además, de las cuatro columnas, tres eran NaN.

Estas variables son biológicamente relevantes, ya que afectan el comportamiento de desplazamiento de los osos polares. Sin embargo, su integración conlleva un coste computacional elevado, especialmente en tareas de enriquecimiento espacio-temporal para grandes volúmenes de datos.

**Por esta razón, se ha decidido posponer su integración hasta futuras versiones del proyecto.**  
Se ha priorizado el análisis del movimiento basado en variables propias del GPS y derivadas internas (distancia, velocidad, aceleración, dirección, etc.), que son más sostenibles en términos de esfuerzo y rendimiento.


## . Guardar dataset final con features

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
df.to_parquet(f"data/processed/feature_dataset_{timestamp}.parquet", index=False)